In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import os
import numpy as np
import time
import os.path as osp
import tensorflow as tf

In [3]:
print(os.getcwd())
os.chdir('/content/drive/MyDrive/MTP/CenterNet')
print(os.getcwd())

/content
/content/drive/MyDrive/MTP/CenterNet


In [4]:
from generators.pascal import PascalVocGenerator
from generators.csv_ import CSVGenerator
from models.resnet import centernet
from generators.utils import affine_transform, get_affine_transform
from eval.common import _get_detections, _get_annotations, evaluate, _compute_ap
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

In [5]:
args_test_annotations_path = 'NUS/valid/valid_annot.csv'   # 'Ouhands_bounding/test_colour/test_annot.csv'
args_classes_path = 'NUS/classes.csv'   # 'Ouhands_bounding/classes.csv'

In [6]:
generator = CSVGenerator(
    args_test_annotations_path,
    args_classes_path,
    shuffle_groups=False,
)

generator_input_size = 512
generator_num_classes = 10
generator_classes = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'G': 6,
    'H': 7,
    'I': 8,
    'J': 9
}
'''generator_classes = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'H': 6,
    'I': 7,
    'J': 8,
    'K': 9
}'''

FileNotFoundError: ignored

In [ ]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
'''generator = PascalVocGenerator(
    'pascal_datasets/test/VOC2007',
    'test',
    shuffle_groups=False,
    skip_truncated=False,
    skip_difficult=True,
)

generator_input_size = 512
generator_num_classes = 20
generator_classes = {
    'aeroplane': 0,
    'bicycle': 1,
    'bird': 2,
    'boat': 3,
    'bottle': 4,
    'bus': 5,
    'car': 6,
    'cat': 7,
    'chair': 8,
    'cow': 9,
    'diningtable': 10,
    'dog': 11,
    'horse': 12,
    'motorbike': 13,
    'person': 14,
    'pottedplant': 15,
    'sheep': 16,
    'sofa': 17,
    'train': 18,
    'tvmonitor': 19
}'''

In [ ]:
'''def preprocess_image(image, c, s, tgt_w, tgt_h):
  trans_input = get_affine_transform(c, s, (tgt_w, tgt_h))
  image = cv2.warpAffine(image, trans_input, (tgt_w, tgt_h), flags=cv2.INTER_LINEAR)
  image = image.astype(np.float32)

  image[..., 0] -= 103.939
  image[..., 1] -= 116.779
  image[..., 2] -= 123.68

  image[..., 0] /= 255.0
  image[..., 1] /= 255.0
  image[..., 2] /= 255.0

  return image'''

In [ ]:
model_path = 'checkpoints/NUS_Transfer_learning_normalization_2_2000_without_bn_with_danet/csv_36_0.9883_0.8095_1.0270.h5'
num_classes = generator_num_classes
classes = list(generator_classes.keys())
flip_test = False
nms = False
keep_resolution = False
score_threshold = 0.2
iou_threshold = 0.5
max_detections = 100
colors = [np.random.randint(0, 256, 3).tolist() for i in range(num_classes)]
model, prediction_model, debug_model = centernet(num_classes=num_classes,
                                                 backbone='resnet101',
                                                 use_triplet_attention=False,
                                                 score_threshold=score_threshold,
                                                 nms=nms,
                                                 flip_test=flip_test
                                                 )
prediction_model.load_weights(model_path, by_name=True)

In [ ]:
# run evaluation
average_precisions, f1_scores, conf_matrix = evaluate(
            generator,
            prediction_model,
            iou_threshold=iou_threshold,
            score_threshold=score_threshold,
            max_detections=max_detections,
            visualize=False
        )


# compute per class average precision
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations),
          generator.label_to_name(label), 'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)

mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
print('mAP: {:.4f}'.format(mean_ap))


# compute per class F1-score
total_instances = []
scores = []
for label, (f1_score, num_annotations) in f1_scores.items():
    print('{:.0f} instances of class'.format(num_annotations),
              generator.label_to_name(label), 'with F1-score: {:.4f}'.format(f1_score))
    total_instances.append(num_annotations)
    scores.append(f1_score)

mean_f1_score = sum(scores) / sum(x > 0 for x in total_instances)
print('Mean F1-score: {:.4f}'.format(mean_f1_score))

cmd_obj = ConfusionMatrixDisplay(conf_matrix, display_labels=generator_classes.keys())
fig, ax = plt.subplots()
#plt.rcParams.update({'font.size': 16})
cmd_obj.plot(ax=ax)
cmd_obj.ax_.set(
                #title='Confusion Matrix', 
                xlabel='Predicted Classes', 
                ylabel='Actual Classes')

In [ ]:
for i in range(generator.size()):
  image = generator.load_image(i)
#path = 'pascal_datasets/test/VOC2007/JPEGImages/009915.jpg'
#image = cv2.imread(path)
  src_image = image.copy()

  c = np.array([image.shape[1] / 2., image.shape[0] / 2.], dtype=np.float32)
  s = max(image.shape[0], image.shape[1]) * 1.0

  tgt_w = generator.input_size
  tgt_h = generator.input_size
  image = generator.preprocess_image(image, c, s, tgt_w=tgt_w, tgt_h=tgt_h)
  if flip_test:
    flipped_image = image[:, ::-1]
    inputs = np.stack([image, flipped_image], axis=0)
  else:
    inputs = np.expand_dims(image, axis=0)

  # run network
  start = time.time()
  detections = prediction_model.predict_on_batch(inputs)[0]
  print("Elapsed time: ", time.time() - start)
  scores = detections[:, 4]

  # sort by score
  indices = np.argsort(-scores)[0]

  if (scores[indices]>=score_threshold):

    '''# select indices which have a score above the threshold
    indices = np.where(scores > score_threshold)'''

    # select those detections
    detections = np.expand_dims(detections[indices], axis=0)

    detections_copy = detections.copy()
    detections = detections.astype(np.float64)
    trans = get_affine_transform(c, s, (tgt_w // 4, tgt_h // 4), inv=1)

    for j in range(detections.shape[0]):
      detections[j, 0:2] = affine_transform(detections[j, 0:2], trans)
      detections[j, 2:4] = affine_transform(detections[j, 2:4], trans)

    detections[:, [0, 2]] = np.clip(detections[:, [0, 2]], 0, src_image.shape[1])
    detections[:, [1, 3]] = np.clip(detections[:, [1, 3]], 0, src_image.shape[0])

    all_detections = []
    for class_id in range(generator.num_classes()):
      class_detections = tf.boolean_mask(detections, tf.equal(detections[:, 5], class_id))
      #print("For class-{} Class detections shape before:".format(class_id), class_detections.shape)
      nms_keep_indices = tf.image.non_max_suppression(tf.cast(class_detections[:, :4],tf.float32),
                                                      tf.cast(class_detections[:, 4],tf.float32),
                                                      5,
                                                      iou_threshold=iou_threshold,
                                                      score_threshold=score_threshold)
      class_detections = tf.gather(class_detections, nms_keep_indices)
      
      if (class_detections.shape[0]>0):
        all_detections.append(class_detections)

    if (all_detections!=[]):  
      all_detections = tf.concat(all_detections, axis=0)
    
    for detection in detections:
        #detection = detections.numpy()
        xmin = int(round(detection[0]))
        ymin = int(round(detection[1]))
        xmax = int(round(detection[2]))
        ymax = int(round(detection[3]))
        score = '{:.4f}'.format(detection[4])
        class_id = int(detection[5])
        color = colors[class_id]
        class_name = classes[class_id]
        label = '-'.join(['Class', class_name])   #'-'.join([class_name, score])
        ret, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.3, 3)
        cv2.rectangle(src_image, (xmin, ymin), (xmax, ymax), color, 3)
        cv2.rectangle(src_image, (xmin, ymax - ret[1] - baseline), (xmin + ret[0], ymax), color, -1)
        cv2.putText(src_image, label, (xmin, ymax - baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)

  #image_fname = 'test_{}'.format(i)
  #cv2.imwrite('test/NUS/{}.jpg'.format(image_fname), src_image)